In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import string

nltk.download('stopwords')
from nltk.corpus import stopwords
sns.set(style='whitegrid')
pd.set_option("display.max_colwidth",200)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df=pd.read_csv('./backend/data/data.csv')
df.head()

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-41419190\r\n,Four ways Bob Corker skewered Donald Trump,"Image copyright Getty Images\nOn Sunday morning, Donald Trump went off on a Twitter tirade against a member of his own party.\nThis, in itself, isn't exactly huge news. It's far from the first tim...",1
1,https://www.reuters.com/article/us-filmfestival-london-lastflagflying/linklaters-war-veteran-comedy-speaks-to-modern-america-says-star-idUSKBN1CD0X2\r\n,"Linklater's war veteran comedy speaks to modern America, says star","LONDON (Reuters) - “Last Flag Flying”, a comedy-drama about Vietnam war veterans, will resonate with Trump’s America, despite, or perhaps because of, its period setting, actor Bryan Cranston said ...",1
2,https://www.nytimes.com/2017/10/09/us/politics/corkers-blast-at-trump-has-other-republicans-nodding-in-agreement.html?rref=collection%2Fsectioncollection%2Fpolitics\r\n,Trump’s Fight With Corker Jeopardizes His Legislative Agenda,The feud broke into public view last week when Mr. Corker said that Mr. Trump’s advisers were guarding against “chaos.” The president retaliated on Sunday by saying the retiring senator “didn’t ha...,1
3,https://www.reuters.com/article/us-mexico-oil-cardenas/egypts-cheiron-wins-tie-up-with-pemex-for-mexican-onshore-oil-field-idUSKBN1C92EU\r\n,Egypt's Cheiron wins tie-up with Pemex for Mexican onshore oil field,"MEXICO CITY (Reuters) - Egypt’s Cheiron Holdings Limited won the rights to partner with Mexican national oil company Pemex on its onshore Cardenas-Mora project, the industry regulator said on Wedn...",1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/snl-jason-aldean-las-vegas-tom-petty-orig.cnn/video/playlists/cant-miss/\n,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performing as the Las Vegas shooting began, did the ""Saturday Night Live"" cold open with a tribute to the victims and their families by singing Tom Petty's ""I ...",1


In [ ]:
df.info()
df.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   URLs      4009 non-null   object
 1   Headline  4009 non-null   object
 2   Body      3988 non-null   object
 3   Label     4009 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 125.4+ KB


,URLs,Headline,Body,Label
count,4009,4009,3988,4009.000000
unique,3352,2831,2863,NaN
top,http://beforeitsnews.com/sports/2017/09/930-expo-park-forbes-field-era-temple-cup-08-race-mo-bros-kiner-wallops-herman-deal-k-mart-2701555.html\r\n,"10/3 Happenings: HBD Cap, Johnny, Dog, Tomato, Ismael, Wil & Alex; Frankie & Gene Hired; Clarke Retires",A Potato Battery Can Light up a Room for Over a Month DIY (System That Can Be Used to Provide Rooms With LED-Powered Lighting for as Long as 40 days) +Video,NaN
freq,5,6,143,NaN
mean,NaN,NaN,NaN,0.466949
std,NaN,NaN,NaN,0.498969
min,NaN,NaN,NaN,0.000000
25%,NaN,NaN,NaN,0.000000
50%,NaN,NaN,NaN,0.000000
75%,NaN,NaN,NaN,1.000000


In [ ]:
df=df.drop_duplicates()
df=df.drop_duplicates(subset=["Body"])
df.shape

(2864, 4)

In [ ]:
df.isnull().sum()

,0
URLs,0
Headline,0
Body,1
Label,0


In [ ]:
df["Headline"]=df["Headline"].fillna("")
df["Body"]=df["Body"].fillna("")

In [ ]:
df["headline_len"]=df["Headline"].apply(len)
df["body_len"]=df["Body"].apply(len)
df["word_count"]=df["Body"].apply(lambda x: len(str(x).split()))
df.head()

,URLs,Headline,Body,Label,headline_len,body_len,word_count
0,http://www.bbc.com/news/world-us-canada-41419190\r\n,Four ways Bob Corker skewered Donald Trump,"Image copyright Getty Images\nOn Sunday morning, Donald Trump went off on a Twitter tirade against a member of his own party.\nThis, in itself, isn't exactly huge news. It's far from the first tim...",1,42,6412,1061
1,https://www.reuters.com/article/us-filmfestival-london-lastflagflying/linklaters-war-veteran-comedy-speaks-to-modern-america-says-star-idUSKBN1CD0X2\r\n,"Linklater's war veteran comedy speaks to modern America, says star","LONDON (Reuters) - “Last Flag Flying”, a comedy-drama about Vietnam war veterans, will resonate with Trump’s America, despite, or perhaps because of, its period setting, actor Bryan Cranston said ...",1,66,3325,551
2,https://www.nytimes.com/2017/10/09/us/politics/corkers-blast-at-trump-has-other-republicans-nodding-in-agreement.html?rref=collection%2Fsectioncollection%2Fpolitics\r\n,Trump’s Fight With Corker Jeopardizes His Legislative Agenda,The feud broke into public view last week when Mr. Corker said that Mr. Trump’s advisers were guarding against “chaos.” The president retaliated on Sunday by saying the retiring senator “didn’t ha...,1,60,6580,1086
3,https://www.reuters.com/article/us-mexico-oil-cardenas/egypts-cheiron-wins-tie-up-with-pemex-for-mexican-onshore-oil-field-idUSKBN1C92EU\r\n,Egypt's Cheiron wins tie-up with Pemex for Mexican onshore oil field,"MEXICO CITY (Reuters) - Egypt’s Cheiron Holdings Limited won the rights to partner with Mexican national oil company Pemex on its onshore Cardenas-Mora project, the industry regulator said on Wedn...",1,68,630,97
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/snl-jason-aldean-las-vegas-tom-petty-orig.cnn/video/playlists/cant-miss/\n,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performing as the Las Vegas shooting began, did the ""Saturday Night Live"" cold open with a tribute to the victims and their families by singing Tom Petty's ""I ...",1,43,213,37


In [ ]:
stop_words=set(stopwords.words('english'))
def clean_text(text):
  text=text.lower()
  text=re.sub(r"http\S+|www\S+","",text)
  text=re.sub(r"<.*?>","",text)
  text=text.translate(str.maketrans("","",string.punctuation))
  text=re.sub(r"\s+"," ",text).strip()
  return text

In [ ]:
df["clean_text"]=df["Headline"]+" "+df["Body"]
df["clean_text"]=df["clean_text"].apply(clean_text)
df.head()

,URLs,Headline,Body,Label,headline_len,body_len,word_count,clean_text
0,http://www.bbc.com/news/world-us-canada-41419190\r\n,Four ways Bob Corker skewered Donald Trump,"Image copyright Getty Images\nOn Sunday morning, Donald Trump went off on a Twitter tirade against a member of his own party.\nThis, in itself, isn't exactly huge news. It's far from the first tim...",1,42,6412,1061,four ways bob corker skewered donald trump image copyright getty images on sunday morning donald trump went off on a twitter tirade against a member of his own party this in itself isnt exactly hu...
1,https://www.reuters.com/article/us-filmfestival-london-lastflagflying/linklaters-war-veteran-comedy-speaks-to-modern-america-says-star-idUSKBN1CD0X2\r\n,"Linklater's war veteran comedy speaks to modern America, says star","LONDON (Reuters) - “Last Flag Flying”, a comedy-drama about Vietnam war veterans, will resonate with Trump’s America, despite, or perhaps because of, its period setting, actor Bryan Cranston said ...",1,66,3325,551,linklaters war veteran comedy speaks to modern america says star london reuters “last flag flying” a comedydrama about vietnam war veterans will resonate with trump’s america despite or perhaps be...
2,https://www.nytimes.com/2017/10/09/us/politics/corkers-blast-at-trump-has-other-republicans-nodding-in-agreement.html?rref=collection%2Fsectioncollection%2Fpolitics\r\n,Trump’s Fight With Corker Jeopardizes His Legislative Agenda,The feud broke into public view last week when Mr. Corker said that Mr. Trump’s advisers were guarding against “chaos.” The president retaliated on Sunday by saying the retiring senator “didn’t ha...,1,60,6580,1086,trump’s fight with corker jeopardizes his legislative agenda the feud broke into public view last week when mr corker said that mr trump’s advisers were guarding against “chaos” the president reta...
3,https://www.reuters.com/article/us-mexico-oil-cardenas/egypts-cheiron-wins-tie-up-with-pemex-for-mexican-onshore-oil-field-idUSKBN1C92EU\r\n,Egypt's Cheiron wins tie-up with Pemex for Mexican onshore oil field,"MEXICO CITY (Reuters) - Egypt’s Cheiron Holdings Limited won the rights to partner with Mexican national oil company Pemex on its onshore Cardenas-Mora project, the industry regulator said on Wedn...",1,68,630,97,egypts cheiron wins tieup with pemex for mexican onshore oil field mexico city reuters egypt’s cheiron holdings limited won the rights to partner with mexican national oil company pemex on its ons...
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/snl-jason-aldean-las-vegas-tom-petty-orig.cnn/video/playlists/cant-miss/\n,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performing as the Las Vegas shooting began, did the ""Saturday Night Live"" cold open with a tribute to the victims and their families by singing Tom Petty's ""I ...",1,43,213,37,jason aldean opens snl with vegas tribute country singer jason aldean who was performing as the las vegas shooting began did the saturday night live cold open with a tribute to the victims and the...


In [ ]:
df=df[df["clean_text"].str.strip()!=""]
df.shape

(2864, 8)

In [ ]:
df.to_csv("./backend/data/clean_fake_news.csv", index=False)
print("Saved clean dataset.")


Saved clean dataset.
